# Workflow 2, Module 1 (condition similarity)

One approach to solving this module is to not define quite so tightly what's going on at the subquestion level in terms of enrichements and archetypes and so on, but simply pass the question to ROBOKOP and let its scoring bring the the best answers to the top.  Here we will use the quick service to start with a disease, find relevant phenotypes, and from there find genetic conditions. The answers will come out ranked by path.

For more details, see the "quick" notebook in greengamma/general.

First, we'll have a quick function that calls the quick service, and some functions for properly creating the question.  Then we'll create the question, run it, and pretty print it for two examples: diabetes and asthma.

In [1]:
#Load some functions for calling the quick service and parsing its output

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from gg_functions import parse_answer, quick, expand

The basic machine question created below goes from a disease to a set of phenotypes to a genetic_condition.  Making the phenotypes a set allows there to be many phenotypes that connect the disease to the condition.

In [2]:
def create_basic_question(disease_id):
    return {
    "machine_question": {
        "nodes": [
            {
                "id": "n0",
                "type": "disease",
                "curie": disease_id
            },
            {
                "id": "n1",
                "type": "gene"
            },
            {
                "id": "n2",
                "type": "chemical_substance",
                "set": True
            },
            {
                "id": "n3",
                "type": "gene"
            }
        ],
        "edges": [
            {
                "id": "e0",
                "source_id": "n0",
                "target_id": "n1"
            },
            {
                "id": "e1",
                "source_id": "n1",
                "target_id": "n2"
            },
            {
                "id": "e2",
                "source_id": "n2",
                "target_id": "n3"
            }
        ]
    }
}

In [3]:
fanconi = 'MONDO:0019391' #fanconi anemia

## What are the Fanconi Genes?

In [4]:
fanconi_expand = expand('disease',fanconi,'gene')
fanconi_genes = parse_answer(fanconi_expand,node_list=['n1'])
print(f"We found {len(fanconi_genes)} genes associated with {fanconi}. Here are the first few:")
fanconi_genes.head()

http://robokop.renci.org:80/api/simple/expand/disease/MONDO:0019391/gene
Return Status: 200
We found 48 genes associated with MONDO:0019391. Here are the first few:


,n1 - id,n1 - name,score
0,HGNC:3584,FANCC,1.777429
1,HGNC:20473,BRIP1,1.767911
2,HGNC:3588,FANCG,1.767064
3,HGNC:9820,RAD51C,1.717592
4,HGNC:3582,FANCA,1.627382


## What genes are chemically similar to the Fanconi genes?

Let's make a question of the type above, and run it.  The max_connectivity option sets a maximum degree for a node in the path, and is used to control the amount of time it takes to run and the specificity of the result. 1000 is a decent across the board value.

In [5]:
machine_question = create_basic_question(fanconi)
answer = quick(machine_question,max_connectivity=1000)

Return Status: 200


We don't want the answer to include already-known FA genes from the list above, so let's filter those out of the results:

In [6]:
df = parse_answer(answer,node_list=['n1','n3'])
v = fanconi_genes['n1 - id '].values
new_df = df[ ~ df['n3 - id '].isin(v) ]

new_df.head(50)

,n1 - id,n1 - name,n3 - id,n3 - name,score
6,HGNC:3436,ERCC4,HGNC:6204,JUN,1.429436
8,HGNC:3588,FANCG,HGNC:11920,FAS,1.376987
9,HGNC:1100,BRCA1,HGNC:959,BAX,1.363620
10,HGNC:3436,ERCC4,HGNC:1784,CDKN1A,1.361439
11,HGNC:3436,ERCC4,HGNC:3433,ERCC1,1.352890
13,HGNC:3588,FANCG,HGNC:4638,GSTP1,1.335340
15,HGNC:1100,BRCA1,HGNC:1784,CDKN1A,1.307594
16,HGNC:3588,FANCG,HGNC:1784,CDKN1A,1.287911
17,HGNC:3588,FANCG,HGNC:959,BAX,1.283034
18,HGNC:3436,ERCC4,HGNC:7218,MPO,1.247543


## What genes are functionally similar to the FA genes, using functions related to FA?

We can potentially sharpen some of these answers if we demand that the processes are actually related to the input disease somehow:

In [7]:
def create_complex_question(disease_id):
    return {
    "machine_question": {
    "nodes": [
            {
                "id": "n0",
                "type": "disease",
                "curie": disease_id
            },
            {
                "id": "n1",
                "type": "gene"
            },
            {
                "id": "n2",
                "type": "chemical_substance",
                "set": True
            },
            {
                "id": "n3",
                "type": "gene"
            }
        ],
        "edges": [
            {
                "id": "e0",
                "source_id": "n0",
                "target_id": "n1"
            },
            {
                "id": "e1",
                "source_id": "n1",
                "target_id": "n2"
            },
            {
                "id": "e2",
                "source_id": "n2",
                "target_id": "n3"
            },
            {
                "id": "e3",
                "source_id": "n0",
                "target_id": "n2"
            }
        ]
    }}

In [8]:
machine_question_2 = create_complex_question(fanconi)
answer_2 = quick(machine_question_2,max_connectivity=1000)

Return Status: 200


In [9]:
df_2 = parse_answer(answer_2,node_list=['n1','n3'])
v = fanconi_genes['n1 - id '].values
new_df_2 = df_2[ ~ df_2['n3 - id '].isin(v) ]

new_df_2.head(50)

,n1 - id,n1 - name,n3 - id,n3 - name,score
0,HGNC:3588,FANCG,HGNC:4641,GSTT1,0.781987
1,HGNC:3585,FANCD2,HGNC:4641,GSTT1,0.749290
2,HGNC:795,ATM,HGNC:4641,GSTT1,0.691356
4,HGNC:18870,POLN,HGNC:4641,GSTT1,0.652560
5,HGNC:3588,FANCG,HGNC:2631,CYP2E1,0.643066
7,HGNC:3585,FANCD2,HGNC:2631,CYP2E1,0.615214
11,HGNC:795,ATM,HGNC:2631,CYP2E1,0.545922
15,HGNC:18870,POLN,HGNC:2631,CYP2E1,0.499625
18,HGNC:3588,FANCG,HGNC:3401,EPHX1,0.436241
19,HGNC:795,ATM,HGNC:3401,EPHX1,0.427038
